In [1]:
import sys
import os
import findspark
import os


In [2]:
USER = os.environ['USER']
print(USER)

r0t00xk


In [3]:
HADOOP_PATH = "/opt/spark-2.3.0-bin-hadoop2.7"
HADOOP_JARS_PATH = HADOOP_PATH + "/jars/"

In [4]:
findspark.init(HADOOP_PATH)
#import pyspark
import pyspark
from pyspark.sql import SparkSession

In [5]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, ArrayType


In [6]:
pyspark_submit_args= "--master yarn \
--deploy-mode client \
--executor-memory 6G \
--driver-memory 5G \
--queue queue2 \
--executor-cores 4 \
--num-executors 20 \
--conf spark.yarn.dist.archives=gs://p13n-storage2/"+USER+"/envs/dev27.zip#ecop27 \
--archives gs://p13n-storage2/r0t00xk/envs/dev27.zip#ecop27 \
--jars "+HADOOP_JARS_PATH+"/datanucleus-api-jdo-3.2.6.jar,"+HADOOP_JARS_PATH+"/datanucleus-rdbms-3.2.9.jar,"+HADOOP_JARS_PATH+"/datanucleus-core-3.2.10.jar,/u/users/r0t00xk/.ivy2/jars/org.apache.spark_spark-sql-kafka-0-10_2.11-2.3.0.jar,/u/users/r0t00xk/.ivy2/jars/org.apache.kafka_kafka-clients-0.10.1.0.jar \
pyspark-shell"

In [7]:
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
os.environ["PYSPARK_PYTHON"] = './ecop27/dev27/bin/python'
os.environ["PYSPARK_DRIVER_PYTHON"] = './ecop27/dev27/bin/python'

In [8]:
schema = StructType([
  StructField("pageType",StringType(),True),
  StructField("pageContext",StringType(),True),
  StructField("deviceType",StringType(),True),
  StructField("details",ArrayType(StringType()),True),
  StructField("timestamp",StringType(),True),
  StructField("reqId",StringType(),True),
  StructField("betaCoeff",StringType(),True),
  ]
)

In [9]:
spark = SparkSession.builder.getOrCreate()

In [10]:
df = spark.readStream.format("kafka").option("kafka.bootstrap.servers","kafka-1054180686-1-1268601033.wus.kafka-v2-sp-ad-server-prod.ms-df-messaging.prod-az-westus-23.prod.us.walmart.net:9092").option("subscribe", "sp_p13n_request_logs").option("multiLine", "true").load().selectExpr("CAST(value AS STRING) as data")

In [11]:
from pyspark.sql.functions import col,from_json
dv = df.select(from_json(col("data"),schema))

AnalysisException: u'Queries with streaming sources must be executed with writeStream.start();;\nkafka'

In [ ]:
consoleoutput = dv.writeStream.format("console").outputMode("append").option("truncate", "false").start().awaitTermination()

In [ ]:
spark.stop()